In [7]:
# N-gram tfidf 
# have not drop stopword 
# have dropping digits
"""
EQB = [ [EQB item] * number of EQB item] 
[EQB item] = [ [paragraph1 in ITB1] , [ paragraph2 in ITB2] , [paragraph3 in ITB3] ] 
"""

#only use EOEG,NC6,JGS,liwa as training data
#use PTT_LNG as valid data 



import pandas as pd
import numpy as np
import nltk
import math
import re

In [8]:
pd_EOEG = pd.read_excel('EQB.xlsx',sheet_name = 'EOEG' );
pd_NC6 = pd.read_excel("EQB.xlsx",sheet_name='NC6');
pd_JGS = pd.read_excel("EQB.xlsx",sheet_name= 'JGS' );
pd_liwa = pd.read_excel("EQB.xlsx",sheet_name= 'LIWA' );
pd_PTT = pd.read_excel("EQB.xlsx",sheet_name= 'PTT LNG' );

In [9]:
EQB = [ [] for i in range(406) ];

def add_EQB(df):
    check = df['paragraph'].isna();
    for i in range(len(df)): 
        if check[i] == False:
            EQB[i].append(df['paragraph'][i])

add_EQB(pd_NC6);
add_EQB(pd_EOEG);
add_EQB(pd_JGS);
add_EQB(pd_liwa);

fuck = pd_NC6['Quality Activity'];

In [10]:
from nltk.stem import SnowballStemmer
import  nltk;
import math;
from nltk.stem import PorterStemmer
stemmer = SnowballStemmer('english')

In [11]:
def getgram(paragraph , gram):
    ret = []
    for i in range( len(paragraph) - gram  ):
        combine = '';
        for j in range(gram):
            combine += paragraph[i+j] + ' '
        ret.append(combine);
    return ret;

test = ['test1' , 'test2' , 'test3' , '11' ,'22' , '33']
print(getgram(test,3))

['test1 test2 test3 ', 'test2 test3 11 ', 'test3 11 22 ']


In [12]:
r1 = u'[’!"#$%&\'()*+,-./:;<=>?@，。?★、…【】《》？“”‘’！[\\]^_`{|}~]+'
def preprocess_paragraph(paragraph):
    raw_words = nltk.word_tokenize(paragraph); #tokenize words by nltk
    words = [stemmer.stem(w) for w in raw_words] # stem
    drop = [] #drop digit
    for w in words:
        w = re.sub(r1,'',w)
        if w.isdigit():
            drop.append('<digit>')
        elif w != '':
            drop.append(w)
        else:
            drop.append('<punct>')
    return drop;


In [13]:
def update_dic(key , dic): #dictnary中令一个key的value+1
    if key not in dic:
        dic[key] = 1;
    else: 
        dic[key] += 1;
    return dic

In [14]:
def get_tf(EQB_field , df , gram):
    num_words = 0;
    freq = {};
    for paragraph in EQB_field:
        words = preprocess_paragraph(paragraph) #just drop 標點符號和數字
        words = getgram(words,gram) # get N-gram words
        num_words += len(words) # calculate the total number of these paragraphs
        for w in words:
            freq = update_dic( key = w , dic = freq) # calculate tf
    tf = {}        
    for (word , count) in freq.items():
        tf[word] = count * 1.0 / num_words; # 修正每个词为正确的频率
        df = update_dic( key = word , dic = df)    #calculate df
    return tf , df;

In [15]:
def get_tfidf(tf_list , df , N):
    tfidf_list = []
    for tf in tf_list:
        tfidf = {}
        for (word , tf_value) in tf.items():
            tfidf[word] = tf_value * math.log( N / ( 1 + df[word]) , 2 ) 
        tfidf_list.append(tfidf);
    return tfidf_list

In [16]:
def solve(EQB , gram):
    df = {}
    tf_list = []
    id_list = []
    N = len(EQB);
    
    item = pd_EOEG['Item'][0]
    paragraph_num = []
    for i in range(len(EQB)):
        EQB_field = EQB[i];
        id = pd_EOEG['Quality Activity'][i] 
        if (pd_EOEG['Item'].isna()[i] == False):
            item = pd_EOEG['Item'][i] 
        
        if len(EQB_field) == 0:
            N = N - 1
            continue    
        tf , df = get_tf(EQB_field, df , gram)
        paragraph_num.append(len(EQB_field))
        tf_list.append(tf)
        id_list.append(item + ' ' + id)
    tfidf_list = get_tfidf(tf_list , df , N)
    
    
    for i in range(len(tfidf_list)):
        tfidf = tfidf_list[i]
        id = id_list[i]
        print(id)
        print('paragraph num = ' , paragraph_num[i])
        s = pd.Series(tfidf)
        print(s.sort_values(ascending=False).head(10))


In [17]:
solve(EQB , gram = 2)

1.1 a. Minimun Vertical Clearance for Access
paragraph num =  4
<punct> walkway      0.157489
walkway <punct>      0.146662
platform <punct>     0.120575
elev platform        0.115166
headroom <punct>     0.104993
work area            0.104993
headroom over        0.104993
<punct> and          0.086090
pipe <punct>         0.072393
<punct> or           0.062992
dtype: float64
1.1 b. Minimun width Clearance for Access
paragraph num =  3
c minimum            0.217116
minimum width        0.197937
width of             0.157859
platform <punct>     0.151543
of walkway           0.108558
oper aisl            0.108558
<punct> structur     0.108558
<digit> equip        0.108558
elev <punct>         0.108558
or elev              0.108558
dtype: float64
1.2 a. Maximum distance between landings on a sectional ladder
paragraph num =  4
platform shall       0.140551
for ladder           0.124163
<punct> platform     0.094445
distanc between      0.082776
meter abov           0.082776
maximum dista

In [18]:
solve(EQB , gram = 3)

1.1 a. Minimun Vertical Clearance for Access
paragraph num =  4
<punct> walkway <punct>      0.163165
elev platform <punct>        0.119316
platform <punct> walkway     0.108776
<punct> <punct> <digit>      0.063686
headroom <punct> <punct>     0.059658
and work area                0.059658
work area <punct>            0.059658
<digit> for overhead         0.059658
for overhead pipe            0.059658
pipe <punct> the             0.059658
dtype: float64
1.1 b. Minimun width Clearance for Access
paragraph num =  3
c minimum width               0.228347
oper aisl <digit>             0.114173
of walkway <punct>            0.114173
structur <punct> platform     0.114173
<punct> structur <punct>      0.114173
equip <punct> structur        0.114173
<digit> equip <punct>         0.114173
elev <punct> <punct>          0.114173
or elev <punct>               0.114173
grade or elev                 0.114173
dtype: float64
1.2 a. Maximum distance between landings on a sectional ladder
paragraph nu

dtype: float64
2.6.2 b. Personnel protection, maximum allowable temp.
paragraph num =  1
<punct> pp <punct>          0.441470
be abov 65c                0.220735
surfac which satisfi        0.220735
personnel protect insul     0.220735
protect insul <punct>       0.220735
insul <punct> pp            0.220735
pp <punct> <digit>          0.220735
<digit> <punct> pp          0.220735
pp <punct> insul            0.220735
<punct> insul shall         0.220735
dtype: float64
2.7 Wind Speed, annual average
paragraph num =  1
blower and air            0.110419
<punct> 40°c •            0.110419
<punct> 19°c •            0.110419
min <punct> 19°c          0.110419
40°c <punct> min          0.110419
<punct> 40°c <punct>      0.110419
max <punct> 40°c          0.110419
condit <punct> max        0.110419
design condit <punct>     0.110419
• design condit           0.110419
dtype: float64
2.8.1 Ambient temperature, annual average
paragraph num =  1
blower and air            0.110419
<punct> 40°c • 